# Python Refresher Lab — Instructor Notebook (Solutions + Rationale)

This notebook contains complete solutions and short rationale (FAANG-style). Runs top-to-bottom.

In [ ]:
from __future__ import annotations
from collections import Counter, defaultdict, deque
import heapq

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

## Section 1 — Python Idioms & Core Data Structures

In [ ]:
# Task 1.1 Solutions
even_squares = [x * x for x in range(1, 21) if x % 2 == 0]
pairs = [(x, x * x) for x in range(1, 11)]
nested = [[1, 2], [3, 4], [5, 6]]
flattened = [v for row in nested for v in row]

check('even_squares', even_squares == [4, 16, 36, 64, 100, 144, 196, 256, 324, 400])
check('pairs', pairs[:3] == [(1, 1), (2, 4), (3, 9)] and pairs[-1] == (10, 100))
check('flattened', flattened == [1, 2, 3, 4, 5, 6])

# Rationale: comprehensions move looping into CPython internals and keep code concise.
# They are not 'magically' faster than all loops, but they reduce Python-level overhead and are idiomatic.

In [ ]:
# Task 1.2 Solutions
s = 'mississippi'
freq = Counter(s)
top2 = freq.most_common(2)
print(freq)
print(top2)
check('top2', top2[0][0] == 'i' and top2[0][1] == 4)

# Rationale: Counter is O(n) counting; sorting characters is O(n log n).
# In interviews, mention that Counter uses hashing and is ideal for frequency-based problems.

In [ ]:
# Task 1.3 Solutions
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry', 'avocado']
groups = defaultdict(list)
for w in words:
    groups[w[0]].append(w)

print(dict(groups))
check('groups[a]', dict(groups)['a'] == ['apple', 'apricot', 'avocado'])

# Rationale: defaultdict avoids repeated key checks and keeps grouping code clean and O(n).

## Section 2 — FAANG Core Patterns

In [ ]:
# Task 2.1 Two Sum
def two_sum(nums, target):
    seen = {}  # value -> index
    for i, x in enumerate(nums):
        y = target - x
        if y in seen:
            return [seen[y], i]
        seen[x] = i
    return []

check('two_sum_1', two_sum([2, 7, 11, 15], 9) == [0, 1])
check('two_sum_2', two_sum([3, 2, 4], 6) == [1, 2])
check('two_sum_dup', two_sum([3, 3], 6) == [0, 1])

# Rationale: one pass + hash lookups => O(n) time, O(n) space.
# Gotcha: duplicates; you must add after checking complement.

In [ ]:
# Task 2.2 Sliding window
def length_of_longest_substring(s: str) -> int:
    last = {}
    l = 0
    best = 0
    for r, ch in enumerate(s):
        if ch in last and last[ch] >= l:
            l = last[ch] + 1
        last[ch] = r
        best = max(best, r - l + 1)
    return best

check('ls_abcabcbb', length_of_longest_substring('abcabcbb') == 3)
check('ls_bbbbb', length_of_longest_substring('bbbbb') == 1)
check('ls_pwwkew', length_of_longest_substring('pwwkew') == 3)
check('ls_empty', length_of_longest_substring('') == 0)

# Rationale: maintain invariant that s[l..r] has unique chars.
# Each index moves forward at most once => O(n) time.

In [ ]:
# Task 2.3 Product Except Self
def product_except_self(nums):
    n = len(nums)
    out = [1] * n
    prefix = 1
    for i in range(n):
        out[i] = prefix
        prefix *= nums[i]
    suffix = 1
    for i in range(n - 1, -1, -1):
        out[i] *= suffix
        suffix *= nums[i]
    return out

check('pes_1234', product_except_self([1, 2, 3, 4]) == [24, 12, 8, 6])
check('pes_zero', product_except_self([-1, 1, 0, -3, 3]) == [0, 0, 9, 0, 0])

# Rationale: output[i] = product(prefix before i) * product(suffix after i).
# Works with zeros because we never divide; zeros naturally propagate through prefix/suffix.

## Section 3 — FAANG Hard (Optional / Bonus)

In [ ]:
def max_sliding_window(nums, k):
    if k <= 0:
        return []
    dq = deque()  # indices, values decreasing
    out = []
    for i, x in enumerate(nums):
        # drop out-of-window
        while dq and dq[0] < i - k + 1:
            dq.popleft()
        # maintain decreasing values
        while dq and nums[dq[-1]] <= x:
            dq.pop()
        dq.append(i)
        if i >= k - 1:
            out.append(nums[dq[0]])
    return out

check('msw', max_sliding_window([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7])

# Rationale: each index is pushed/popped at most once => amortized O(n).